## RAG Evalutation 

### Evaluation Strategy: 

For every query, our retreiver will retrieve top 3,5,7 documents. From our query-corpus dataset, we will match if the documents retreived match/are in the groud truth document mapped to a query.

### Retreiver Strategy:
1. We will use embedding based retreival techniques - text-embedding-ada-002 from GenAI hub as well as keyword based retreivels like BM25. 

2. We will use open-source BGE re-ranking model to augment the retreival performance

3. We will try Fusion ( index & query re-writing) to see if there is an increase in the performance

4. We are not evaluating the cost/latency for this POC which could be influcened by CPUs/ data size. 

5. Vector Store: I'm using FAISS vector store which is one of the most efficient & fastest technique and uses ANN for indexing the document. 

6. Framework : using LlamaIndex framework

#### Defining llm & embedding

In [ ]:
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.langchain.proxy import OpenAIEmbeddings
from ipywidgets import widgets

llm_model_name = widgets.Dropdown(
    options=[
        "gpt-35-turbo",
        "gpt-35-turbo-16k",
        "gpt-4",
        "gpt-4-32k",
        "gpt-4-turbo",
        "gemini-1.0-pro",
        "gpt-4-vision"
        # "tiiuae--falcon-40b-instruct"
    ],
    value="gpt-35-turbo-16k",
    description="LLM Model Name",
    disabled=False,
)


llm = ChatOpenAI(proxy_model_name=llm_model_name.value)
embeddings = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002')

from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.langchain import LangChainLLM

llama_llm = LangChainLLM(llm)
llama_emb= LangchainEmbedding(embeddings)

from llama_index.core import Settings
Settings.embed_model = embeddings
Settings.llm = llm

#### Loading open source SciFact Dataset from BIER

In [5]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader

C:\Users\I068117\Anaconda3\envs\myenv10\lib\site-packages\beir\util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
#### Download scifact.zip dataset and unzip the dataset
import logging
import pathlib, os
dataset = "scifact"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(pathlib.Path("C:/Users/I068117/UT_Machine Learning/Custom-AI-Chatbot-RAG").parent.absolute(), "datasets")
data_path = util.download_and_unzip(url, out_dir)

#### Provide the data_path where scifact has been downloaded and unzipped
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

#### Each corpus id is a single document, which we will use for our evaluation. There are 5183 documents in the corpus with a mean chunk size of 1400 words

We are not doing any chunking as each document is chunked as per the corpus id

Later, we will explore combining the whole document into one document and then chunking it.

In [8]:
# Read the JSON Lines file
df_corpus = pd.read_json('C:/Users/I068117/UT_Machine Learning/datasets/scifact/corpus.jsonl', lines=True)
df_corpus.head()

,_id,title,text,metadata
0,4983,Microstructural development of human newborn c...,Alterations of the architecture of cerebral wh...,{}
1,5836,Induction of myelodysplasia by myeloid-derived...,Myelodysplastic syndromes (MDS) are age-depend...,{}
2,7912,"BC1 RNA, the transcript from a master gene for...",ID elements are short interspersed elements (S...,{}
3,18670,The DNA Methylome of Human Peripheral Blood Mo...,DNA methylation plays an important role in bio...,{}
4,19238,The human myelin basic protein gene is include...,Two human Golli (for gene expressed in the oli...,{}


#### Current chunk size for the documents 

In [345]:
text_length= df_corpus['text'].apply(lambda x : len(x)).mean()
print(text_length)

1401.0839282268955


#### Queries df - There are 1109 queries

In [9]:
# Read the JSON Lines file
df_queries = pd.read_json('C:/Users/I068117/UT_Machine Learning/datasets/scifact/queries.jsonl', lines=True)
df_queries.head()

,_id,text,metadata
0,0,0-dimensional biomaterials lack inductive prop...,{}
1,2,1 in 5 million in UK have abnormal PrP positiv...,"{'13734012': [{'sentences': [4], 'label': 'CON..."
2,4,1-1% of colorectal cancer patients are diagnos...,{}
3,6,10% of sudden infant death syndrome (SIDS) dea...,{}
4,9,32% of liver transplantation programs required...,"{'44265107': [{'sentences': [15], 'label': 'SU..."


In [346]:
len(df_queries)

1109

#### Each Query is mapped to a corpus-id document as the ground truth. There are 919 queries mapped to a corpus id in the train Q&A set. 

In [7]:
import pandas as pd
pd.set_option('display.max_rows', None)
df=pd.read_csv('C:/Users/I068117/UT_Machine Learning/datasets/scifact/qrels/train.tsv',sep="\t")
df.head()

,query-id,corpus-id,score
0,0,31715818,1
1,2,13734012,1
2,4,22942787,1
3,6,2613775,1
4,9,44265107,1


##### Getting unique query id with all relevant documents as one query could have 1 or more relevant documents 

In [ ]:
df_key = df.groupby('query-id')['corpus-id'].agg(list).reset_index()

In [85]:
#------------optional to load embeddings & storing locally---------------
new=[]
for d in data:
    new.append(embeddings.embed_query(d.page_content))
df_corpus['emb']=new

#df_corpus.to_excel('scifact_emd.xlsx')

#### LLama_Index

In [101]:
import json
data_Json = []

with open("corpus.jsonl", "r") as file:
    for line in file:
        data_Json.append(json.loads(line))

In [45]:
# Write the list of JSON objects to a JSON file
with open('corpus.json', 'w') as json_file:
    json.dump(data_Json, json_file, indent=2)

In [11]:
from llama_index.core import (
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display

In [12]:
import faiss
d = 1536
faiss_index = faiss.IndexFlatL2(d)

In [100]:
#converting document from Langchain to Llama_index document
from llama_index.core import Document
docs_llama=[]
for doc in docs_langchain:
    docs_llama.append(Document.from_langchain_format(doc))

In [131]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    docs_llama, storage_context=storage_context
)

In [132]:
#Storing index on the disk
index.storage_context.persist(persist_dir="C:/Users/I068117/UT_Machine Learning/Custom-AI-Chatbot-RAG")

### VectorStore Retreiver

In [15]:
from llama_index.core import StorageContext, load_index_from_storage

# load index from disk
vector_store = FaissVectorStore.from_persist_dir("C:/Users/I068117/UT_Machine Learning/Custom-AI-Chatbot-RAG")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="C:/Users/I068117/UT_Machine Learning/Custom-AI-Chatbot-RAG"
)

#loading the index from the local directory to avoid loading the embeddings
index = load_index_from_storage(storage_context=storage_context)
#context = vector_retriever.retrieve("A deficiency of vitamin B12 decreases blood levels of homocysteine")
#print(context)

In [82]:
def vector_retriever_corpus_index(x,topk):
    vector_retriever = index.as_retriever(similarity_top_k=topk)
    context = vector_retriever.retrieve(x)
    ci=[]
    for c in context:
        ci.append(c.node.metadata['idx'])
    return ci

In [83]:
# Function to calculate recall
def calculate_recall(row,k):
    matches = len(set(list(map(str,row['corpus-id']))) & set(list(row[f'top{k}'])))
    return matches / len(row['corpus-id'])

In [89]:
# Function to calculate recall
def calculate_precision(row,k):
    matches = len(set(list(map(str,row['corpus-id']))) & set(list(row[f'top{k}'])))
    return matches / len(row[f'top{k}'])

In [94]:
from typing import List

def calculate_reciprocal_rank(row, k):
    mrr = []
    for idx in row['corpus-id']:
        if str(idx) not in row[f'top{k}']:
            mrr.append(0)
        else:
            mrr.append(1 / (row[f'top{k}'].index(str(idx)) + 1))
    return sum(mrr) / len(mrr)


In [84]:
df_eval=pd.DataFrame(columns=['_id','query'])
df_eval[['_id','query']]= df_queries.iloc[:100,:2]
df_eval_merge= df_eval.merge(df_key,left_on='_id',right_on='query-id',how='left')

In [86]:
df_eval_merge['top3']=df_eval_merge['query'].apply(lambda x: vector_retriever_corpus_index(x,topk=3))
df_eval_merge['top5']=df_eval_merge['query'].apply(lambda x:vector_retriever_corpus_index(x,topk=5))
df_eval_merge['top7']=df_eval_merge['query'].apply(lambda x:vector_retriever_corpus_index(x,topk=7))

In [ ]:
df_eval_merge['recall@3'] = df_eval_merge.apply(lambda x:calculate_recall(x,k=3), axis=1)
df_eval_merge['recall@5'] = df_eval_merge.apply(lambda x:calculate_recall(x,k=5), axis=1)
df_eval_merge['recall@7'] = df_eval_merge.apply(lambda x:calculate_recall(x,k=7), axis=1)

df_eval_merge['MRR@3'] = df_eval_merge.apply(lambda x:calculate_reciprocal_rank(x,k=3), axis=1)
df_eval_merge['MRR@5'] = df_eval_merge.apply(lambda x:calculate_reciprocal_rank(x,k=5), axis=1)
df_eval_merge['MRR@7'] = df_eval_merge.apply(lambda x:calculate_reciprocal_rank(x,k=7), axis=1)


In [315]:
eval_df = pd.DataFrame([{
    'Retreival_Technique': 'text-embedding-ada-002',
    'Recall@3': round(df_eval_merge['recall@3'].mean(), 2),
    'Recall@5': round(df_eval_merge['recall@5'].mean(), 2),
    'Recall@7': round(df_eval_merge['recall@7'].mean(), 2),
    'MRR@3': round(df_eval_merge['MRR@3'].mean(), 2),
    'MRR@5': round(df_eval_merge['MRR@5'].mean(), 2),
    'MRR@7': round(df_eval_merge['MRR@7'].mean(), 2),
    'HT@3': round(df_eval_merge[df_eval_merge['recall@3'] != 0].shape[0] / len(df_eval_merge), 2),
    'HT@5': round(df_eval_merge[df_eval_merge['recall@5'] != 0].shape[0] / len(df_eval_merge), 2),
    'HT@7': round(df_eval_merge[df_eval_merge['recall@7'] != 0].shape[0] / len(df_eval_merge), 2)
}])
eval_df

,Retreival_Technique,Recall@3,Recall@5,Recall@7,MRR@3,MRR@5,MRR@7,HT@3,HT@5,HT@7
0,text-embedding-ada-002,0.72,0.82,0.84,0.66,0.69,0.69,0.73,0.84,0.86


### Vectorstore retreiver with a re-ranker(top 5)

In [303]:
vector_rerank_retriever = index.as_retriever(similarity_top_k=10)

In [304]:
df_vector_rerank_eval=pd.DataFrame(columns=['_id','query'])
df_vector_rerank_eval[['_id','query']]= df_queries.iloc[:100,:2]
df_vector_rerank_eval_merge= df_vector_rerank_eval.merge(df_key,left_on='_id',right_on='query-id',how='left')

In [305]:
df_vector_rerank_eval_merge['top5']=df_vector_rerank_eval_merge['query'].apply(lambda x:re_ranker(vector_rerank_retriever.retrieve(x),x))

In [ ]:
df_vector_rerank_eval_merge['recall@5'] = df_vector_rerank_eval_merge.apply(lambda x:calculate_recall(x,k=5), axis=1)
df_vector_rerank_eval_merge['MRR@5'] = df_vector_rerank_eval_merge.apply(lambda x:calculate_reciprocal_rank(x,k=5), axis=1)

In [321]:
new_data = {
    'Retreival_Technique': 'text-embedding-ada-002 + BGE Re_ranker_top5',
    'Recall@5': round(df_vector_rerank_eval_merge['recall@5'].mean() , 2),  # Adjust these as needed
    'MRR@5': round(df_vector_rerank_eval_merge['MRR@5'].mean(), 2),
    'HT@5': round(df_vector_rerank_eval_merge[df_vector_rerank_eval_merge['recall@5'] != 0].shape[0] / len(df_vector_rerank_eval_merge), 2),
}

eval_df.loc[len(eval_df)] = new_data

eval_df 

,Retreival_Technique,Recall@3,Recall@5,Recall@7,MRR@3,MRR@5,MRR@7,HT@3,HT@5,HT@7
0,text-embedding-ada-002,0.72,0.82,0.84,0.66,0.69,0.69,0.73,0.84,0.86
1,text-embedding-ada-002 + BGE Re_ranker_top5,NaN,0.82,NaN,NaN,0.67,NaN,NaN,0.85,NaN


### BM25 retreiver

In [113]:
# We can pass in the index, doctore, or list of nodes to create the retriever
from llama_index.retrievers.bm25 import BM25Retriever
BM25_retriever = BM25Retriever.from_defaults(nodes=docs_llama, similarity_top_k=7)
def BM25_retriever_corpus_index(x):
    context = BM25_retriever.retrieve(x)
    ci=[]
    for c in context:
        ci.append(c.node.metadata['idx'])
    return ci


In [102]:
df_BM_eval=pd.DataFrame(columns=['_id','query'])
df_BM_eval[['_id','query']]= df_queries.iloc[:100,:2]
df_BM_eval_merge= df_BM_eval.merge(df_key,left_on='_id',right_on='query-id',how='left')

In [108]:
df_BM_eval_merge['top3']=df_BM_eval_merge['query'].apply(lambda x: BM25_retriever_corpus_index(x))

In [110]:
df_BM_eval_merge['top5']=df_BM_eval_merge['query'].apply(lambda x:BM25_retriever_corpus_index(x))

In [114]:
df_BM_eval_merge['top7']=df_BM_eval_merge['query'].apply(lambda x:BM25_retriever_corpus_index(x))

In [ ]:
# Apply function to each row
df_BM_eval_merge['recall@3'] = df_BM_eval_merge.apply(lambda x:calculate_recall(x,k=3), axis=1)
df_BM_eval_merge['recall@5'] = df_BM_eval_merge.apply(lambda x:calculate_recall(x,k=5), axis=1)
df_BM_eval_merge['recall@7'] = df_BM_eval_merge.apply(lambda x:calculate_recall(x,k=7), axis=1)

In [ ]:
df_BM_eval_merge['MRR@3'] = df_BM_eval_merge.apply(lambda x:calculate_reciprocal_rank(x,k=3), axis=1)
df_BM_eval_merge['MRR@5'] = df_BM_eval_merge.apply(lambda x:calculate_reciprocal_rank(x,k=5), axis=1)
df_BM_eval_merge['MRR@7'] = df_BM_eval_merge.apply(lambda x:calculate_reciprocal_rank(x,k=7), axis=1)

In [323]:
new_data = {
    'Retreival_Technique': 'BM25 Retreiver',
    'Recall@3': round(df_BM_eval_merge['recall@3'].mean(), 2),
    'Recall@5': round(df_BM_eval_merge['recall@5'].mean(), 2),
    'Recall@7': round(df_BM_eval_merge['recall@7'].mean(), 2),
    'MRR@3': round(df_BM_eval_merge['MRR@3'].mean(), 2),
    'MRR@5': round(df_BM_eval_merge['MRR@5'].mean(), 2),
    'MRR@7': round(df_BM_eval_merge['MRR@7'].mean(), 2),
    'HT@3': round(df_BM_eval_merge[df_BM_eval_merge['recall@3'] != 0].shape[0] / len(df_BM_eval_merge), 2),
    'HT@5': round(df_BM_eval_merge[df_BM_eval_merge['recall@5'] != 0].shape[0] / len(df_BM_eval_merge), 2),
    'HT@7': round(df_BM_eval_merge[df_BM_eval_merge['recall@7'] != 0].shape[0] / len(df_BM_eval_merge), 2)
}

eval_df.loc[len(eval_df)] = new_data

eval_df

,Retreival_Technique,Recall@3,Recall@5,Recall@7,MRR@3,MRR@5,MRR@7,HT@3,HT@5,HT@7
0,text-embedding-ada-002,0.72,0.82,0.84,0.66,0.69,0.69,0.73,0.84,0.86
1,text-embedding-ada-002 + BGE Re_ranker_top5,NaN,0.82,NaN,NaN,0.67,NaN,NaN,0.85,NaN
2,BM25 Retreiver,0.63,0.68,0.70,0.49,0.50,0.50,0.64,0.69,0.71


### BM25 with a re-ranker

In [298]:
BM25_retriever = BM25Retriever.from_defaults(nodes=docs_llama, similarity_top_k=10)

In [299]:
df_BM25_rerank_eval=pd.DataFrame(columns=['_id','query'])
df_BM25_rerank_eval[['_id','query']]= df_queries.iloc[:100,:2]
df_BM25_rerank_eval_merge= df_BM25_rerank_eval.merge(df_key,left_on='_id',right_on='query-id',how='left')

In [300]:
df_BM25_rerank_eval_merge['top5']=df_hybrid_rerank_eval_merge['query'].apply(lambda x:re_ranker(BM25_retriever.retrieve(x),x))

In [ ]:
df_BM25_rerank_eval_merge['recall@5'] = df_BM25_rerank_eval_merge.apply(lambda x:calculate_recall(x,k=5), axis=1)
df_BM25_rerank_eval_merge['MRR@5'] = df_BM25_rerank_eval_merge.apply(lambda x:calculate_reciprocal_rank(x,k=5), axis=1)

In [331]:
new_data = {
    'Retreival_Technique': 'BM25 Retreiver + BGE Re_ranker_top5',
    'Recall@5': round(df_BM25_rerank_eval_merge['recall@5'].mean() , 2),  # Adjust these as needed
    'MRR@5': round(df_BM25_rerank_eval_merge['MRR@5'].mean(), 2),
    'HT@5': round(df_BM25_rerank_eval_merge[df_BM25_rerank_eval_merge['recall@5'] != 0].shape[0] / len(df_BM25_rerank_eval_merge), 2),
}

eval_df.loc[3] = new_data

eval_df 

,Retreival_Technique,Recall@3,Recall@5,Recall@7,MRR@3,MRR@5,MRR@7,HT@3,HT@5,HT@7
0,text-embedding-ada-002,0.72,0.82,0.84,0.66,0.69,0.69,0.73,0.84,0.86
1,text-embedding-ada-002 + BGE Re_ranker_top5,NaN,0.82,NaN,NaN,0.67,NaN,NaN,0.85,NaN
2,BM25 Retreiver,0.63,0.68,0.70,0.49,0.50,0.50,0.64,0.69,0.71
3,BM25 Retreiver + BGE Re_ranker_top5,NaN,0.71,NaN,NaN,0.61,NaN,NaN,0.73,NaN
4,Hybrid reteiver - text-ada+BM25),NaN,0.77,0.84,NaN,NaN,NaN,NaN,0.77,0.86
5,Hybrid retreiver + BGE Re_ranker_top5,NaN,0.82,NaN,NaN,0.68,NaN,NaN,0.84,NaN


### Hybrid custom retreiver ( Vectorstore+BM25)

In [256]:
from llama_index.core.retrievers import BaseRetriever
                                             
class HybridRetriever(BaseRetriever):
    def __init__(self, vector_retriever, bm25_retriever):
        self.vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        super().__init__()

    def _retrieve(self, query):
        bm25_nodes = self.bm25_retriever.retrieve(query)
        vector_nodes = self.vector_retriever.retrieve(query)

       # combine the two lists of nodes
        all_nodes = []
        node_ids = set()
        for n in bm25_nodes + vector_nodes:
            if n.node.node_id not in node_ids:
                all_nodes.append(n)
                node_ids.add(n.node.node_id)
        return all_nodes


In [279]:
vector_retriever = index.as_retriever(similarity_top_k=5)
bm25_retriever = BM25Retriever.from_defaults(nodes=docs_llama, similarity_top_k=5)

hybrid_retriever = HybridRetriever(vector_retriever, bm25_retriever)

In [271]:
df_hybrid2_eval=pd.DataFrame(columns=['_id','query'])
df_hybrid2_eval[['_id','query']]= df_queries.iloc[:100,:2]
df_hybrid2_eval_merge= df_hybrid2_eval.merge(df_key,left_on='_id',right_on='query-id',how='left')

In [273]:
def get_index(all_nodes):
    ci=[]
    for c in all_nodes:
        ci.append(c.node.metadata['idx'])
    return ci

In [276]:
df_hybrid2_eval_merge['top5']=df_hybrid2_eval_merge['query'].apply(lambda x:get_index(hybrid_retriever.retrieve(x)))

In [280]:
df_hybrid2_eval_merge['top10']=df_hybrid2_eval_merge['query'].apply(lambda x:get_index(hybrid_retriever.retrieve(x)))

In [ ]:
df_hybrid2_eval_merge['recall@5'] = df_hybrid2_eval_merge.apply(lambda x:calculate_recall(x,k=5), axis=1)
df_hybrid2_eval_merge['recall@10'] = df_hybrid2_eval_merge.apply(lambda x:calculate_recall(x,k=10), axis=1)

In [328]:
new_data = {
    'Retreival_Technique': 'Hybrid reteiver - text-ada+BM25)',
    'Recall@5': round(df_hybrid2_eval_merge['recall@5'].mean() , 2),  # Adjust these as needed
    'Recall@7': round(df_hybrid2_eval_merge['recall@10'].mean(), 2),
    'HT@5': round(df_hybrid2_eval_merge[df_hybrid2_eval_merge['recall@5'] != 0].shape[0] / len(df_hybrid2_eval_merge), 2),
    'HT@7': round(df_hybrid2_eval_merge[df_hybrid2_eval_merge['recall@10'] != 0].shape[0] / len(df_hybrid2_eval_merge), 2),
}

eval_df.loc[len(eval_df)] = new_data

eval_df 

,Retreival_Technique,Recall@3,Recall@5,Recall@7,MRR@3,MRR@5,MRR@7,HT@3,HT@5,HT@7
0,text-embedding-ada-002,0.72,0.82,0.84,0.66,0.69,0.69,0.73,0.84,0.86
1,text-embedding-ada-002 + BGE Re_ranker_top5,NaN,0.82,NaN,NaN,0.67,NaN,NaN,0.85,NaN
2,BM25 Retreiver,0.63,0.68,0.70,0.49,0.50,0.50,0.64,0.69,0.71
3,text-embedding-ada-002 + BGE Re_ranker_top5,NaN,0.71,NaN,NaN,0.61,NaN,NaN,0.73,NaN
4,Hybrid reteiver - text-ada+BM25),NaN,0.77,0.84,NaN,NaN,NaN,NaN,0.77,0.86


### Hybrid custom Retreiver with Re-ranker

In [ ]:
from llama_index.core.postprocessor import SentenceTransformerRerank
reranker = SentenceTransformerRerank(top_n=5, model="BAAI/bge-reranker-base")

In [287]:
df_hybrid_rerank_eval=pd.DataFrame(columns=['_id','query'])
df_hybrid_rerank_eval[['_id','query']]= df_queries.iloc[:100,:2]
df_hybrid_rerank_eval_merge= df_hybrid_rerank_eval.merge(df_key,left_on='_id',right_on='query-id',how='left')

In [294]:
from llama_index.core import QueryBundle
from llama_index.core.schema import NodeWithScore

def re_ranker(retrieved_nodes,query):
    reranked_nodes = reranker.postprocess_nodes(retrieved_nodes,
                                                query_bundle=QueryBundle(query),)
    index_id = []
    index_id = get_index(reranked_nodes)
    return index_id

In [295]:
df_hybrid_rerank_eval_merge['top5']=df_hybrid_rerank_eval_merge['query'].apply(lambda x:re_ranker(hybrid_retriever.retrieve(x),x))

In [ ]:
df_hybrid_rerank_eval_merge['recall@5'] = df_hybrid_rerank_eval_merge.apply(lambda x:calculate_recall(x,k=5), axis=1)
df_hybrid_rerank_eval_merge['MRR@5'] = df_hybrid_rerank_eval_merge.apply(lambda x:calculate_reciprocal_rank(x,k=5), axis=1)

In [330]:
new_data = {
    'Retreival_Technique': 'Hybrid retreiver + BGE Re_ranker_top5',
    'Recall@5': round(df_hybrid_rerank_eval_merge['recall@5'].mean() , 2),  # Adjust these as needed
    'MRR@5': round(df_hybrid_rerank_eval_merge['MRR@5'].mean(), 2),
    'HT@5': round(df_hybrid_rerank_eval_merge[df_hybrid_rerank_eval_merge['recall@5'] != 0].shape[0] / len(df_hybrid_rerank_eval_merge), 2),
}

eval_df.loc[len(eval_df)-1] = new_data

eval_df 

,Retreival_Technique,Recall@3,Recall@5,Recall@7,MRR@3,MRR@5,MRR@7,HT@3,HT@5,HT@7
0,text-embedding-ada-002,0.72,0.82,0.84,0.66,0.69,0.69,0.73,0.84,0.86
1,text-embedding-ada-002 + BGE Re_ranker_top5,NaN,0.82,NaN,NaN,0.67,NaN,NaN,0.85,NaN
2,BM25 Retreiver,0.63,0.68,0.70,0.49,0.50,0.50,0.64,0.69,0.71
3,text-embedding-ada-002 + BGE Re_ranker_top5,NaN,0.71,NaN,NaN,0.61,NaN,NaN,0.73,NaN
4,Hybrid reteiver - text-ada+BM25),NaN,0.77,0.84,NaN,NaN,NaN,NaN,0.77,0.86
5,Hybrid retreiver + BGE Re_ranker_top5,NaN,0.82,NaN,NaN,0.68,NaN,NaN,0.84,NaN


### Hybrid Fusion retreiver

In [167]:
from llama_index.core.tools import RetrieverTool
index = load_index_from_storage(storage_context=storage_context)
vector_retriever = index.as_retriever(similarity_top_k=3)
bm25_retriever = BM25Retriever.from_defaults(nodes=docs_llama, similarity_top_k=3)

retriever_tools = [
    RetrieverTool.from_defaults(
        retriever=vector_retriever,
        description="Useful in most cases",
    ),
    RetrieverTool.from_defaults(
        retriever=bm25_retriever,
        description="Useful if searching about specific information",
    ),
]

In [142]:
from llama_index.core.retrievers import RouterRetriever

retriever = RouterRetriever.from_defaults(
    retriever_tools=retriever_tools,
    llm=llama_llm,
    select_multi=True,
)

def hybrid_retriever_corpus_index(x):
    context = retriever.retrieve(x)
    ci=[]
    for c in context:
        ci.append(c.node.metadata['idx'])
    return ci

In [136]:
df_hybrid_eval=pd.DataFrame(columns=['_id','query'])
df_hybrid_eval[['_id','query']]= df_queries.iloc[:100,:2]
df_hybrid_eval_merge= df_hybrid_eval.merge(df_key,left_on='_id',right_on='query-id',how='left')

In [ ]:
df_hybrid_eval_merge['top5']=df_hybrid_eval_merge['query'].apply(lambda x:hybrid_retriever_corpus_index(x))

In [155]:
df_hybrid_eval_merge['recall@5'] = df_hybrid_eval_merge.apply(lambda x:calculate_recall(x,k=5), axis=1)
print(df_hybrid_eval_merge['recall@5'].mean())

0.785


In [ ]:
new_data = {
    'Retreival_Technique': 'Hybrid retreiver + BGE Re_ranker_top5',
    'Recall@5': round(df_hybrid_rerank_eval_merge['recall@5'].mean() , 2),  # Adjust these as needed
    'MRR@5': round(df_hybrid_rerank_eval_merge['MRR@5'].mean(), 2),
    'HT@5': round(df_hybrid_rerank_eval_merge[df_hybrid_rerank_eval_merge['recall@5'] != 0].shape[0] / len(df_hybrid_rerank_eval_merge), 2),
}

eval_df.loc[len(eval_df)-1] = new_data

eval_df 

### Simple Fusion Retreiver (Query rewriting): combine retrieval results from multiple queries and multiple indexes.

In [119]:
from llama_index.core import VectorStoreIndex

index_1 = load_index_from_storage(storage_context=storage_context)
index_2 = load_index_from_storage(storage_context=storage_context)

In [130]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [index_1.as_retriever(), index_2.as_retriever()],
    similarity_top_k=7,
    num_queries=4,  # set this to 1 to disable query generation
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

def Fusion_vector_retriever_corpus_index(x):
    context = retriever.retrieve(x)
    ci=[]
    for c in context:
        ci.append(c.node.metadata['idx'])
    return ci

In [122]:
df_fusion_eval=pd.DataFrame(columns=['_id','query'])
df_fusion_eval[['_id','query']]= df_queries.iloc[:100,:2]
df_fusion_eval_merge= df_fusion_eval.merge(df_key,left_on='_id',right_on='query-id',how='left')

In [ ]:
# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

df_fusion_eval_merge['top3']=df_fusion_eval_merge['query'].apply(lambda x: Fusion_vector_retriever_corpus_index(x))

In [ ]:
df_fusion_eval_merge['top5']=df_fusion_eval_merge['query'].apply(lambda x: Fusion_vector_retriever_corpus_index(x))

In [ ]:
df_fusion_eval_merge['top7']=df_fusion_eval_merge['query'].apply(lambda x: Fusion_vector_retriever_corpus_index(x))

In [ ]:
df_fusion_eval_merge['recall@3'] = df_fusion_eval_merge.apply(lambda x:calculate_recall(x,k=3), axis=1)
df_fusion_eval_merge['recall@5'] = df_fusion_eval_merge.apply(lambda x:calculate_recall(x,k=5), axis=1)
df_fusion_eval_merge['recall@7'] = df_fusion_eval_merge.apply(lambda x:calculate_recall(x,k=7), axis=1)

In [332]:
new_data = {
    'Retreival_Technique': 'Simple Fusion Retreiver',
    'Recall@3': round(df_fusion_eval_merge['recall@3'].mean(), 2),
    'Recall@5': round(df_fusion_eval_merge['recall@5'].mean(), 2),
    'Recall@7': round(df_fusion_eval_merge['recall@7'].mean(), 2),
    'HT@3': round(df_fusion_eval_merge[df_fusion_eval_merge['recall@3'] != 0].shape[0] / len(df_fusion_eval_merge), 2),
    'HT@5': round(df_fusion_eval_merge[df_fusion_eval_merge['recall@5'] != 0].shape[0] / len(df_fusion_eval_merge), 2),
    'HT@7': round(df_fusion_eval_merge[df_fusion_eval_merge['recall@7'] != 0].shape[0] / len(df_fusion_eval_merge), 2)
}

eval_df.loc[len(eval_df)] = new_data

,Retreival_Technique,Recall@3,Recall@5,Recall@7,MRR@3,MRR@5,MRR@7,HT@3,HT@5,HT@7
0,text-embedding-ada-002,0.72,0.82,0.84,0.66,0.69,0.69,0.73,0.84,0.86
1,text-embedding-ada-002 + BGE Re_ranker_top5,NaN,0.82,NaN,NaN,0.67,NaN,NaN,0.85,NaN
2,BM25 Retreiver,0.63,0.68,0.70,0.49,0.50,0.50,0.64,0.69,0.71
3,BM25 Retreiver + BGE Re_ranker_top5,NaN,0.71,NaN,NaN,0.61,NaN,NaN,0.73,NaN
4,Hybrid reteiver - text-ada+BM25),NaN,0.77,0.84,NaN,NaN,NaN,NaN,0.77,0.86
5,Hybrid retreiver + BGE Re_ranker_top5,NaN,0.82,NaN,NaN,0.68,NaN,NaN,0.84,NaN
6,Simple Fusion Retreiver,0.61,0.81,0.83,NaN,NaN,NaN,0.63,0.82,0.84


In [334]:
# After replacing NAN with '-' : 
eval_df.fillna('_',inplace=True)

,Retreival_Technique,Recall@3,Recall@5,Recall@7,MRR@3,MRR@5,MRR@7,HT@3,HT@5,HT@7
0,text-embedding-ada-002,0.72,0.82,0.84,0.66,0.69,0.69,0.73,0.84,0.86
1,text-embedding-ada-002 + BGE Re_ranker_top5,-,0.82,-,-,0.67,-,-,0.85,-
2,BM25 Retreiver,0.63,0.68,0.7,0.49,0.5,0.5,0.64,0.69,0.71
3,BM25 Retreiver + BGE Re_ranker_top5,-,0.71,-,-,0.61,-,-,0.73,-
4,Hybrid reteiver - text-ada+BM25),-,0.77,0.84,-,-,-,-,0.77,0.86
5,Hybrid retreiver + BGE Re_ranker_top5,-,0.82,-,-,0.68,-,-,0.84,-
6,Simple Fusion Retreiver,0.61,0.81,0.83,-,-,-,0.63,0.82,0.84


## Trying with different chunking strategies

In [ ]:
from llama_index.readers.json import JSONReader
reader = JSONReader()
documents = reader.load_data(input_file="corpus.json", extra_info={})

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)
nodes[0]

## Langchain

In [99]:
from langchain.docstore.document import Document
import json
docs_langchain=[]
        # Load JSON file
with open("corpus.json") as file:
    data = json.load(file)

    for item in data:
        index = item['_id']
        title = item['title']
        text = item['text']
        metadata = dict(idx =index, title= title, extra = item['metadata'])
        docs_langchain.append(Document(page_content=text, metadata=metadata))

In [125]:
docs_langchain[0]

Document(page_content='Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 versus 1.1 microm2/ms). Relative anisotropy was higher the closer birth was 

In [70]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(docs_langchain[:1000], embeddings)

In [72]:
retriever = vectorstore.as_retriever()

In [123]:
result = retriever.invoke("RANK-RANKL pathway signalling has no known association with development of Aire-expressing medullary thymic epithelial cells.")
result

[Document(page_content='Medullary thymic epithelial cells (mTECs) establish T cell self-tolerance through the expression of autoimmune regulator (Aire) and peripheral tissue-specific self-antigens. However, signals underlying mTEC development remain largely unclear. Here, we demonstrate crucial regulation of mTEC development by receptor activator of NF-kappaB (RANK) and CD40 signals. Whereas only RANK signaling was essential for mTEC development during embryogenesis, in postnatal mice, cooperation between CD40 and RANK signals was required for mTEC development to successfully establish the medullary microenvironment. Ligation of RANK or CD40 on fetal thymic stroma in vitro induced mTEC development in a tumor necrosis factor-associated factor 6 (TRAF6)-, NF-kappaB inducing kinase (NIK)-, and IkappaB kinase beta (IKKbeta)-dependent manner. These results show that developmental-stage-dependent cooperation between RANK and CD40 promotes mTEC development, thereby establishing self-tolerance

In [104]:
result = vectorstore.similarity_search_with_score("RANK-RANKL pathway signalling has no known association with development of Aire-expressing medullary thymic epithelial cells.")